<a href="https://colab.research.google.com/github/shivansh1325/Sign-language_to_text_prediction_deep_Learning/blob/main/sign_language_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input, BatchNormalization
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import SGD
import warnings

warnings.filterwarnings("ignore")

In [ ]:
def load_image(image_path, img_size=(64, 64)):
    image = load_img(image_path, target_size=img_size)
    image = img_to_array(image)
    return image

In [2]:
!pip install kaggle


In [3]:
!kaggle datasets download -d harshvardhan21/sign-language-detection-using-images

Dataset URL: https://www.kaggle.com/datasets/harshvardhan21/sign-language-detection-using-images
License(s): apache-2.0
 95% 255M/268M [00:01<00:00, 218MB/s]
100% 268M/268M [00:01<00:00, 207MB/s]


In [4]:
!unzip sign-language-detection-using-images.zip -d /content/sign-language-detection


Streaming output truncated to the last 5000 lines.
  inflating: /content/sign-language-detection/data/V/819.jpg  
  inflating: /content/sign-language-detection/data/V/82.jpg  
  inflating: /content/sign-language-detection/data/V/820.jpg  
  inflating: /content/sign-language-detection/data/V/821.jpg  
  inflating: /content/sign-language-detection/data/V/822.jpg  
  inflating: /content/sign-language-detection/data/V/823.jpg  
  inflating: /content/sign-language-detection/data/V/824.jpg  
  inflating: /content/sign-language-detection/data/V/825.jpg  
  inflating: /content/sign-language-detection/data/V/826.jpg  
  inflating: /content/sign-language-detection/data/V/827.jpg  
  inflating: /content/sign-language-detection/data/V/828.jpg  
  inflating: /content/sign-language-detection/data/V/829.jpg  
  inflating: /content/sign-language-detection/data/V/83.jpg  
  inflating: /content/sign-language-detection/data/V/830.jpg  
  inflating: /content/sign-language-detection/data/V/831.jpg  
  infl

In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define the dataset path
data_path = '/content/sign-language-detection/data'

# Initialize lists for images and labels
images = []
labels = []

# Loop through each subfolder (representing a label) and load the images
for label in os.listdir(data_path):
    label_path = os.path.join(data_path, label)
    for image_file in os.listdir(label_path):
        image_path = os.path.join(label_path, image_file)
        # Read the image and resize it
        image = cv2.imread(image_path)
        image = cv2.resize(image, (32, 32))  # Resize to 32x32 pixels
        images.append(image)
        labels.append(label)

# Convert lists to NumPy arrays
images = np.array(images)
labels = np.array(labels)

# Encode labels (categorical)
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)
labels_one_hot = to_categorical(labels_encoded)

# Normalize the image data
images = images / 255.0

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(images, labels_one_hot, test_size=0.2, random_state=42)

# Data Augmentation on the training data using ImageDataGenerator
datagen = ImageDataGenerator(
    rotation_range=20,        # Random rotation between 0 and 20 degrees
    width_shift_range=0.2,    # Random horizontal shifts
    height_shift_range=0.2,   # Random vertical shifts
    shear_range=0.2,          # Shear transformations
    zoom_range=0.2,           # Random zooms
    horizontal_flip=True,     # Random horizontal flips
    fill_mode='nearest'       # Fill pixels that are lost after transformation
)

# Fit the ImageDataGenerator on the training data
datagen.fit(X_train)

# Build the CNN model
model = Sequential()

# First Convolutional Layer
model.add(Conv2D(64, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Second Convolutional Layer
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Third Convolutional Layer
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Flatten the data for Dense layers
model.add(Flatten())

# Fully connected layer with 256 units
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))

# Output layer for classification
model.add(Dense(len(np.unique(labels_encoded)), activation='softmax'))

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# # Use Early Stopping and Model Checkpointing
# from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
# model_checkpoint = ModelCheckpoint('best_model.h5', monitor='val_accuracy', save_best_only=True)

# # Train the model using data augmentation
# history = model.fit(datagen.flow(X_train, y_train, batch_size=32),
#                     validation_data=(X_test, y_test),
#                     epochs=30,
#                     callbacks=[early_stopping, model_checkpoint])

history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=40, batch_size=32)



/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/40
1050/1050 ━━━━━━━━━━━━━━━━━━━━ 11s 7ms/step - accuracy: 0.5815 - loss: 1.6306 - val_accuracy: 1.0000 - val_loss: 0.0021
Epoch 2/40
1050/1050 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.9959 - loss: 0.0202 - val_accuracy: 1.0000 - val_loss: 2.0735e-04
Epoch 3/40
1050/1050 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.9978 - loss: 0.0085 - val_accuracy: 1.0000 - val_loss: 8.7302e-05
Epoch 4/40
1050/1050 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.9989 - loss: 0.0039 - val_accuracy: 1.0000 - val_loss: 1.1900e-04
Epoch 5/40
1050/1050 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.9992 - loss: 0.0038 - val_accuracy: 1.0000 - val_loss: 1.3317e-05
Epoch 6/40
1050/1050 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.9993 - loss: 0.0029 - val_accuracy: 1.0000 - val_loss: 8.0495e-06
Epoch 7/40
1050/1050 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.9999 - loss: 7.6379e-04 - val_accuracy: 1.0000 - val_loss: 3.5104e-05
Epoch 8/40
1050/1050 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accur

In [ ]:
# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_acc}")


263/263 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 1.0000 - loss: 0.0000e+00
Test Accuracy: 1.0


In [ ]:
# Save the model
model.save('sign_language_model.h5')

# Download the model from Colab
from google.colab import files
files.download('sign_language_model.h5')


import joblib
# Save the label encoder
joblib.dump(label_encoder, 'label_encoder.pkl')

# Download the label encoder
files.download('label_encoder.pkl')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Save the model
model.save('sign_language_model.h5')

# Download the model from Colab
from google.colab import files
files.download('sign_language_model.h5')


import joblib
# Save the label encoder
joblib.dump(label_encoder, 'label_encoder.pkl')

# Download the label encoder
files.download('label_encoder.pkl')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## **Final Model starting from hear**

In [6]:
import os
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

# Path to the downloaded Kaggle dataset
data_dir = '/content/sign-language-detection/data'

# Load and preprocess the images
def load_data(data_dir):
    X, y = [], []

    # List the class folders in the dataset directory
    if not os.path.exists(data_dir):
        raise ValueError(f"Dataset directory '{data_dir}' not found!")

    classes = os.listdir(data_dir)

    if len(classes) == 0:
        raise ValueError(f"No class folders found in '{data_dir}'!")

    print(f"Found classes: {classes}")  # Debugging

    # Create a label map to convert class names into numerical labels
    label_map = {class_name: idx for idx, class_name in enumerate(classes)}

    # Iterate through each class folder
    for class_name in classes:
        class_dir = os.path.join(data_dir, class_name)

        if not os.path.isdir(class_dir):
            print(f"Skipping '{class_dir}' because it's not a directory")
            continue

        # Iterate through the images in each class folder
        for img_name in os.listdir(class_dir):
            img_path = os.path.join(class_dir, img_name)

            # Check if the file is an image (e.g., jpg, jpeg, png)
            if img_name.lower().endswith(('.png', '.jpg', '.jpeg')):
                img = cv2.imread(img_path)

                if img is not None:
                    img = cv2.resize(img, (64, 64))  # Resize image
                    X.append(img)
                    y.append(label_map[class_name])
                else:
                    print(f"Warning: Failed to load image {img_path}")
            else:
                print(f"Skipping file '{img_name}' because it's not a valid image format")

    # Convert to numpy arrays
    X = np.array(X)
    y = np.array(y)

    # Check if any images were loaded
    if len(X) == 0 or len(y) == 0:
        raise ValueError("No images were loaded. Please check your dataset.")

    # Normalize images
    X = X / 255.0

    # One-hot encode the labels
    y = to_categorical(y, num_classes=len(classes))

    print(f"Loaded {len(X)} images in total.")

    return X, y, classes

# Load the dataset
X, y, classes = load_data(data_dir)

# Check if the dataset is non-empty
if len(X) == 0 or len(y) == 0:
    print("Error: No images loaded. Please check your dataset structure and contents.")
else:
    # Split the dataset into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    print(f"Training data shape: {X_train.shape}")
    print(f"Testing data shape: {X_test.shape}")
    print(f"Number of classes: {len(classes)}")


Found classes: ['1', '4', 'N', 'A', 'F', 'V', 'Z', '6', 'D', '2', 'B', 'G', 'L', 'K', 'P', 'R', 'Q', 'Y', 'S', 'J', 'X', 'H', 'T', 'O', 'I', 'M', '5', 'U', '9', 'C', '7', '3', 'W', 'E', '8']
Loaded 42000 images in total.
Training data shape: (33600, 64, 64, 3)
Testing data shape: (8400, 64, 64, 3)
Number of classes: 35


In [7]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Build the CNN model
model = Sequential()

# Add convolutional layers
model.add(Conv2D(64, (3, 3), activation='relu', input_shape=(64, 64, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Flatten the data and add Dense layers
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))

# Output layer (number of classes)
model.add(Dense(len(classes), activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=20, batch_size=32)


Epoch 1/20
1050/1050 [==============================] - 50s 47ms/step - loss: 0.1156 - accuracy: 0.9676 - val_loss: 8.8181e-06 - val_accuracy: 1.0000
Epoch 2/20
1050/1050 [==============================] - 46s 43ms/step - loss: 0.0098 - accuracy: 0.9969 - val_loss: 3.4656e-06 - val_accuracy: 1.0000
Epoch 3/20
1050/1050 [==============================] - 46s 43ms/step - loss: 0.0058 - accuracy: 0.9981 - val_loss: 1.7725e-08 - val_accuracy: 1.0000
Epoch 4/20
1050/1050 [==============================] - 46s 44ms/step - loss: 0.0027 - accuracy: 0.9992 - val_loss: 4.7400e-09 - val_accuracy: 1.0000
Epoch 5/20
1050/1050 [==============================] - 45s 43ms/step - loss: 0.0066 - accuracy: 0.9980 - val_loss: 1.2594e-06 - val_accuracy: 1.0000
Epoch 6/20
1050/1050 [==============================] - 45s 43ms/step - loss: 0.0035 - accuracy: 0.9993 - val_loss: 1.0218e-09 - val_accuracy: 1.0000
Epoch 7/20
1050/1050 [==============================] - 45s 43ms/step - loss: 0.0042 - accuracy: 0.9

In [8]:
# Save the model to a file
model.save('sign_language_model.h5')
